In [77]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier


In [146]:
#load processed, X, y, test
file_loc = '/Users/gregscanlon/Documents/Kaggle/titanic/'

X = pd.read_excel(file_loc+'X_processed.xlsx',index='PassengerId')
y = pd.read_excel(file_loc+'y_processed.xlsx',index='PassengerId')
test = pd.read_excel(file_loc+'gs_test_processed.xlsx',index='PassengerId')

y.set_index('PassengerId',inplace=True)
X.set_index('PassengerId',inplace=True)
test.set_index('PassengerId',inplace=True)

#scale data manually - not to gaussian
#adj_fare, Age to 0-1 scale
X['fare_norm'] = X['adj_fare'].divide(X['adj_fare'].max())
X['age_norm'] = X['Age'].divide(X['Age'].max())

X.drop(['adj_fare','Age'],axis=1,inplace=True)

test['fare_norm'] = test['adj_fare'].divide(test['adj_fare'].max())
test['age_norm'] = test['Age'].divide(test['Age'].max())

test.drop(['adj_fare','Age'],axis=1,inplace=True)

In [94]:
X.head()

,Pclass,m_f,embark_num,name_len,tix_group,rare_title,family_size_cat_single,family_size_cat_small,family_size_cat_medium,family_size_cat_large,age_cat_baby,age_cat_child,age_cat_young_adult,age_cat_mid_life,age_cat_old,fare_norm,age_norm
PassengerId,,,,,,,,,,,,,,,,,
1,3,0,1,0.280488,1,0,0,1,0,0,0,0,1,0,0,0.032690,0.2750
2,1,1,2,0.621951,1,0,0,1,0,0,0,0,0,1,0,0.321416,0.4750
3,3,1,1,0.268293,1,0,1,0,0,0,0,0,0,1,0,0.035734,0.3250
4,1,1,1,0.536585,2,0,0,1,0,0,0,0,0,1,0,0.119714,0.4375
5,3,0,1,0.292683,1,0,1,0,0,0,0,0,0,1,0,0.036297,0.4375


In [7]:
#logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_scaled, y['Survived'])

preds = clf.predict(test_scaled)

/Users/gregscanlon/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
preds_df = pd.DataFrame(preds,index=test.index,columns=['Survived'])
preds_df

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [12]:
preds_df.to_csv(file_loc+'predictions.csv')

Random Forest by sex and class

In [147]:
#Best male models by pclass
#M1: RF - 15, in_leaf 5 = 80-100%
#M2: RF - 50, min_leaf 5 = 95%
#M3: LogReg = 85-90%
preds_dict_male = {}
preds_list_male = []

X_male = X[X.m_f==0]
y_male = y.loc[X_male.index]
y_test_male = test[test.m_f==0]


for pclass in [1,2,3]:
    X_experiment = X_male[X_male.Pclass==pclass]
    y_experiment = y_male.loc[X_experiment.index]
    y_test_exp = y_test_male[y_test_male.Pclass==pclass]

    
    if pclass == 1:
        clf = RandomForestClassifier(n_estimators=15,min_samples_leaf = 5).fit(X_experiment, y_experiment['Survived'])
        
        
    if pclass == 2:
        clf = RandomForestClassifier(n_estimators=50,min_samples_leaf = 5).fit(X_experiment, y_experiment['Survived'])

    if pclass == 3:
        clf = LogisticRegression(random_state=0,solver='lbfgs').fit(X_experiment, y_experiment['Survived'])


    preds_list_male.append(pd.DataFrame(clf.predict(y_test_exp),index = y_test_exp.index))
    #test = test.join(pd.DataFrame(clf.predict(y_test_exp),index = y_test_exp.index))


In [148]:
#Best female models by pclass
#F1: KNN - 14: 95-100%
#F2: 
#F3: 
gs_preds_df = pd.DataFrame()
preds_list_fem = []
X_female = X[X.m_f==1]
y_female = y.loc[X_female.index]
y_test_female = test[test.m_f==1]


for pclass in [1,2,3]:

    X_experiment = X_female[X_female.Pclass==pclass]
    y_experiment = y_female.loc[X_experiment.index]
    y_test_exp = y_test_female.loc[y_test_female.Pclass==pclass]

    if pclass == 1:
        clf = KNeighborsClassifier(n_neighbors=i).fit(X_experiment, y_experiment['Survived'])


    if pclass == 2:
        clf = KNeighborsClassifier(n_neighbors=i).fit(X_experiment, y_experiment['Survived'])
        
    if pclass == 3:
        clf = KNeighborsClassifier(n_neighbors=i).fit(X_experiment, y_experiment['Survived'])
    
    
    preds_list_fem.append(pd.DataFrame(clf.predict(y_test_exp),index = y_test_exp.index))
    #test = test.join(pd.DataFrame(clf.predict(y_test_exp),index = y_test_exp.index))


In [152]:
preds_final.columns

RangeIndex(start=0, stop=1, step=1)

In [154]:
#concat all dfs into one
preds_male = pd.concat(preds_list_male)
preds_female = pd.concat(preds_list_fem)
pred_list = [preds_male,preds_female]
preds_final = pd.concat(pred_list)
preds_final.rename(columns={0:'Survived'},inplace=True)

preds_final.to_csv(file_loc+'multi_mod_preds.csv')
preds_final

,Survived
PassengerId,
903,0
912,0
915,0
920,1
926,1
...,...
1275,1
1300,1
1301,1


In [129]:
gs = test.join(preds_dict_male[1])

In [131]:
gs.loc[903]

Pclass                    1.000000
m_f                       0.000000
embark_num                1.000000
name_len                  0.412698
tix_group                 1.000000
rare_title                0.000000
family_size_cat_single    1.000000
family_size_cat_small     0.000000
family_size_cat_medium    0.000000
family_size_cat_large     0.000000
age_cat_baby              0.000000
age_cat_child             0.000000
age_cat_young_adult       0.000000
age_cat_mid_life          0.000000
age_cat_old               1.000000
fare_norm                 0.050749
age_norm                  0.605263
0                         0.000000
Name: 903, dtype: float64

In [130]:
preds_dict_male[1]

,0
PassengerId,
903,0
912,0
915,1
920,1
926,1
933,1
938,0
942,1
956,1


In [110]:
test['preds'] = pd.DataFrame(data = preds_dict_male[1][0], index = preds_dict_male[1][1])

In [111]:
test['preds']

PassengerId
892    NaN
893    NaN
894    NaN
895    NaN
896    NaN
        ..
1305   NaN
1306   NaN
1307   NaN
1308   NaN
1309   NaN
Name: preds, Length: 418, dtype: float64

In [100]:
preds_dict_male

{1: [array([0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
         0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
         0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]),
  Int64Index([ 903,  912,  915,  920,  926,  933,  938,  942,  956,  959,  960,
               965,  967,  973,  974,  986, 1010, 1023, 1034, 1036, 1038, 1040,
              1050, 1058, 1069, 1073, 1083, 1088, 1094, 1097, 1107, 1109, 1126,
              1128, 1134, 1137, 1144, 1158, 1162, 1179, 1182, 1185, 1190, 1198,
              1200, 1208, 1215, 1219, 1223, 1227, 1247, 1264, 1270, 1282, 1295,
              1296, 1299],
             dtype='int64', name='PassengerId')],
 2: [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  Int64Index([ 894,  899,  905,  908,  922,  923,  943,  946,  953,  970,  976,
               981,

In [78]:
X_male = X[X.m_f==0]
y_male = y.loc[X_male.index]
y_test_male = test[test.m_f==0]

preds_df = y_test_male.copy()
preds_df['Survived'] = 0
preds_dict = {}
rf_dict = {}

for pclass in [1,2,3]:
    print('PClass: ',pclass)
    for i in [14]:
        X_experiment = X_male[X_male.Pclass==pclass]
        y_experiment = y_male.loc[X_experiment.index]
        y_test_exp = y_test_male[y_test_male.Pclass==pclass]

        #clf = RandomForestClassifier(n_estimators=i,min_samples_leaf = 5).fit(X_experiment, y_experiment['Survived'])
        clf = KNeighborsClassifier(n_neighbors=i).fit(X_experiment, y_experiment['Survived'])
        
        rf_dict[pclass] = clf
        
        print('k = ',i,' /train score: ',clf.score(X_experiment, y_experiment['Survived']))
        
        preds_dict[pclass] = [clf.predict(y_test_exp),y_test_exp.index]
        
        #preds_df['Survived'].loc[y_test_exp.index] = preds
    

PClass:  1
k =  14  /train score:  0.7622950819672131
PClass:  2
k =  14  /train score:  0.9166666666666666
PClass:  3
k =  14  /train score:  0.8760806916426513


In [79]:
rf_dict[1].predict(np.array(y_test_male.loc[1088]).reshape(1, -1))

array([0])

In [80]:
rf_dict[2].predict(np.array(y_test_male.loc[981]).reshape(1, -1))

array([1])

In [81]:
rf_dict[3].predict(np.array(y_test_male.loc[1093]).reshape(1, -1))

array([0])

In [85]:
y_test_male[y_test_male.Pclass==1]

,Pclass,Age,m_f,embark_num,name_len,tix_group,adj_fare,rare_title,family_size_cat_single,family_size_cat_small,family_size_cat_medium,family_size_cat_large,age_cat_baby,age_cat_child,age_cat_young_adult,age_cat_mid_life,age_cat_old
PassengerId,,,,,,,,,,,,,,,,,
903,1,46.00,0,1,0.412698,1,26.000000,0,1,0,0,0,0,0,0,0,1
912,1,55.00,0,2,0.349206,1,59.400000,0,0,1,0,0,0,0,0,0,1
915,1,21.00,0,2,0.492063,1,61.379200,0,0,1,0,0,0,0,1,0,0
920,1,41.00,0,1,0.365079,1,30.500000,0,1,0,0,0,0,0,0,1,0
926,1,30.00,0,2,0.380952,2,28.875000,0,0,1,0,0,0,0,0,1,0
933,1,38.92,0,1,0.428571,1,26.550000,0,1,0,0,0,0,0,0,1,0
938,1,45.00,0,2,0.380952,1,29.700000,0,1,0,0,0,0,0,0,1,0
942,1,24.00,0,1,0.380952,2,30.000000,0,0,1,0,0,0,0,1,0,0
956,1,13.00,0,2,0.428571,5,52.475000,0,0,0,1,0,0,1,0,0,0


In [84]:

print('survival rates - overall')
for i in [1,2,3]:
    print('Pclass: ',i,' = ', preds_dict[i][0].sum()/len(preds_dict[i][0]))

survival rates - overall
Pclass:  1  =  0.24561403508771928
Pclass:  2  =  0.015873015873015872
Pclass:  3  =  0.04794520547945205


In [64]:
preds_dict[3]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Int64Index([ 892,  895,  897,  901,  902,  909,  913,  917,  919,  921,
             ...
             1281, 1284, 1286, 1288, 1290, 1291, 1305, 1307, 1308, 1309],
            dtype='int64', name='PassengerId', length=146)]

In [69]:
clf.predict(np.array(y_test_male.loc[1309]).reshape(1, -1))

array([0])

In [43]:
preds_df['Survived'][preds_df['Survived']==1]

PassengerId
915     1
920     1
926     1
933     1
938     1
942     1
956     1
960     1
965     1
972     1
981     1
1010    1
1036    1
1038    1
1040    1
1050    1
1053    1
1073    1
1083    1
1084    1
1086    1
1088    1
1093    1
1097    1
1126    1
1128    1
1144    1
1158    1
1173    1
1182    1
1190    1
1198    1
1199    1
1208    1
1215    1
1223    1
1227    1
1236    1
1296    1
Name: Survived, dtype: int64

In [31]:
pd.DataFrame.from_dict(preds_dict)

ValueError: arrays must all be same length

In [15]:
rf_preds_df = pd.DataFrame(rf_preds,index=test.index,columns=['Survived'])

In [25]:
rf_preds_df.to_csv(file_loc+'rf_predictions.csv')

In [26]:
rf_preds_df

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0
